In [1]:
import numpy as np
import pandas as pd
import random
import torch
from torch.utils.data import Dataset, DataLoader
from torch import nn
from tqdm import tqdm
import time

In [2]:
class NeuMF(nn.Module):
    def __init__(self, num_factors, num_users, num_items, nums_hiddens):
        super(NeuMF, self).__init__()
        self.P = nn.Embedding(num_users, num_factors)
        self.Q = nn.Embedding(num_items, num_factors)
        self.U = nn.Embedding(num_users, num_factors)
        self.V = nn.Embedding(num_items, num_factors)
        self.mlp = nn.Sequential()
        pre_num_hiddens = num_factors*2
        for i in range(0,len(nums_hiddens)):
            self.mlp.add_module('mlp'+str(i),
                nn.Sequential(
                    nn.Linear(pre_num_hiddens,nums_hiddens[i]),
                    nn.ReLU()
                )
            )
            pre_num_hiddens = nums_hiddens[i]
        self.prediction_layer = nn.Sequential(
            nn.Linear(pre_num_hiddens+num_factors,1,bias=False),
            nn.Sigmoid()
        )

    def forward(self, user_id, item_id):
        p_mf = self.P(user_id)
        q_mf = self.Q(item_id)
        gmf = p_mf * q_mf
        p_mlp = self.U(user_id) # (b,1)->(b,num_factors)
        q_mlp = self.V(item_id) # (b,1)->(b,num_factors)
        mlp = self.mlp(torch.concat([p_mlp, q_mlp], dim=1)) # (b,num_factors*2)->(b,last_num_hiddens)
        con_res = torch.concat([gmf, mlp], dim=1) # (b,last_num_hiddens+num_factors)
        return self.prediction_layer(con_res) # (b,last_num_hiddens+num_factors)->(b,1)

In [3]:
class PRDataset(Dataset):
    def __init__(self, users, inter, num_items):
        """

        :param users:
        :param items:
        :param inter: 字典 key为user，value为按时间排序的评价过的item
        :param num_items:
        """
        self.users = users
        self.inter = inter
        self.all = set([i for i in range(num_items)])

    def __len__(self):
        return len(self.users)

    def __getitem__(self, idx):
        neg_items = list(self.all - set(self.inter[int(self.users[idx])]))
        pos_items = self.inter[int(self.users[idx])]
        neg_indices = random.randint(0, len(neg_items) - 1)
        pos_indices = random.randint(0, len(pos_items) - 1)
        return self.users[idx], pos_items[pos_indices], neg_items[neg_indices]

In [4]:
class ArrayDataset(Dataset):
    def __init__(self, ArrayData,transform=None):
        self._data  = ArrayData

    def __len__(self):
        return len(self._data[0])

    def __getitem__(self, idx):
        return tuple(data[idx] for data in self._data)

In [5]:
def hit_and_auc(rankedlist, test_matrix, k):
    """
    计算每个用户的命中数和 AUC。
    :param rankedlist:
    :param test_matrix: 用户真正交互的item
    :param k:
    :return:
    """
    hits_k = [(idx, val) for idx, val in enumerate(rankedlist[:k])
              if val in set(test_matrix)]
    hits_all = [(idx, val) for idx, val in enumerate(rankedlist)
                if val in set(test_matrix)]
    max = len(rankedlist) - 1
    auc = 1.0 * (max - hits_all[0][0]) / max if len(hits_all) > 0 else 0
    return len(hits_k), auc

#@save
def evaluate_ranking(net, test_input, candidates,num_users, num_items,device):
    ranked_list, ranked_items, hit_rate, auc = {}, {}, [], []
    all_items = set([i for i in range(num_items)])
    for u in tqdm(range(num_users)):
        neg_items = list(all_items - set(candidates[int(u)]))
        user_ids, item_ids, x, scores = [], [], [], []
        [item_ids.append(i) for i in neg_items] #记录u没有评价的item id
        [user_ids.append(u) for _ in neg_items] #u的id，和item_ids长度相同
        x.extend([np.array(user_ids)])
        x.extend([np.array(item_ids)])
        # x[0]:len=len(neg_items) 元素全为u,x[1]:len=len(neg_items) 元素为neg_items的id
        x = np.array(x)
        x = torch.tensor(x)
        test_data_set = ArrayDataset(x)
        test_data_iter = DataLoader(test_data_set, shuffle=False, batch_size=1024)
        for user_id, item_id in test_data_iter:
            user_id = user_id.to(device)
            item_id = item_id.to(device)
            score = net(user_id,item_id)
            scores.extend(score)
        scores = [item for sublist in scores for item in sublist]
        item_scores = list(zip(item_ids, scores))
        ranked_list[u] = sorted(item_scores, key=lambda t: t[1], reverse=True)
        ranked_items[u] = [r[0] for r in ranked_list[u]]
        true_item = int(test_input[test_input.u==u+1].i)-1
        temp = hit_and_auc(ranked_items[u], [true_item], 50)
        hit_rate.append(temp[0])
        auc.append(temp[1])
    return np.mean(np.array(hit_rate)), np.mean(np.array(auc))

#@save
def train_ranking(net, train_iter, test_input, optimizer,loss,num_users,num_items, num_epochs, device, evaluator,candidates):
    net = net.to(device)
    print("training on ", device)
    plt_epoch = []
    for epoch in range(num_epochs):
        plt_epoch.append(epoch)
        train_l_sum, train_acc_sum, n, batch_count, start = 0.0, 0.0, 0, 0, time.time()
        for u, pos_item,neg_item in tqdm(train_iter):
            u = u.to(device)
            pos_item = pos_item.to(device)
            neg_item = neg_item.to(device)

            p_pos = net(u,pos_item)
            p_neg = net(u,neg_item)
            l = loss(p_pos, p_neg)
            train_l_sum += l.sum().cpu().item()
            optimizer.zero_grad()
            l.backward()
            optimizer.step()
            batch_count+=1
        with torch.no_grad():
            hit_rate, auc = evaluator(net, test_input,candidates,num_users, num_items,device)
        print('epoch %d,train_loss %.4f,hit_rate %.4f,auc %.4f, time %.1f sec'
              % (epoch + 1,train_l_sum / batch_count,hit_rate,auc, time.time() - start))


In [6]:
def read_data_ml100k():
    names = ['user_id', 'item_id', 'rating', 'timestamp']
    data = pd.read_csv('../../data/ml-100k/u.data', '\t', names=names,engine='python')
    num_users = data.user_id.unique().shape[0]
    num_items = data.item_id.unique().shape[0]
    return data, num_users, num_items

def split_data_ml100k(data, num_users, split_mode='random', test_ratio=0.1):
    """Split the dataset in random mode or seq-aware mode."""
    if split_mode == 'seq-aware':
        train_items, test_items, train_list = {}, {}, []
        for line in data.itertuples():
            u, i, rating, time = line[1], line[2], line[3], line[4]
            train_items.setdefault(u, []).append((u, i, rating, time)) # 如果键不在字典里，setdefault将键和默认值添加到字典中，最后返回该键对应的值。这里返回list，通过append将所有相同user的item放到同一个key对应的value里。
            if u not in test_items or test_items[u][-1] < time: # 将最新的item放到test_items
                test_items[u] = (i, rating, time)
        for u in range(1, num_users + 1):
            train_list.extend(sorted(train_items[u], key=lambda k: k[3])) # 将每个user对应的value 按照时间戳从小到大排序放到train_list
        test_data = [(key, *value) for key, value in test_items.items()] # 将test_items变成list
        train_data = [item for item in train_list if item not in test_data] #将train_list不在test_data里的元素放到train_data
        train_data = pd.DataFrame(train_data,columns=['u', 'i', 'rating', 'time'])
        test_data = pd.DataFrame(test_data,columns=['u', 'i', 'rating', 'time'])
    else:
        mask = np.random.uniform(0, 1, (len(data))) < (1 - test_ratio)# 生成(len(data),)大小的bool类型数组 随机test_ratio比例的元素为False，其余为True
        neg_mask = [not x for x in mask] # 生成len(data)长度的bool类型list 元素和mask相反
        train_data, test_data = data[mask], data[neg_mask]
    return train_data, test_data

def load_data_ml100k(data, num_users, num_items, feedback='explicit'):
    users, items, scores = [], [], []
    inter = np.zeros((num_items, num_users)) if feedback == 'explicit' else {}
    for line in data.itertuples():
        user_index, item_index = int(line[1] - 1), int(line[2] - 1) #0~942 0~1681
        score = int(line[3]) if feedback == 'explicit' else 1
        users.append(user_index)
        items.append(item_index)
        scores.append(score)
        if feedback == 'implicit':
            inter.setdefault(user_index, []).append(item_index) # 隐式则为字典 key为user，value为按时间排序的item list
        else:
            inter[item_index, user_index] = score
    return users, items, scores, inter

In [7]:
df, num_users, num_items = read_data_ml100k()
print(num_users,num_items)
train_data, test_data = split_data_ml100k(df, num_users,'seq-aware')
users_train, items_train, ratings_train, candidates = load_data_ml100k(train_data, num_users, num_items, feedback="implicit")
# 因为使用train_data ，candidates 不包括user最新交互的item
def BPRLoss(positive,negative):
    sigmoid = nn.Sigmoid()
    return - torch.sum(torch.log(sigmoid(positive - negative)), dim=0, keepdim=True)
net = NeuMF(10,num_users, num_items, nums_hiddens=[10, 10, 10])
loss = BPRLoss
optimizer = torch.optim.Adam(net.parameters(), lr=0.01)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
num_epoch = 2
train_set = PRDataset(users_train, candidates, num_items)
train_iter = DataLoader(train_set,batch_size=128,shuffle=True)
evaluator = evaluate_ranking
train_ranking(net, train_iter, test_data, optimizer,loss,num_users,num_items, num_epoch, device, evaluator,candidates)

C:\Users\12548\AppData\Local\Temp/ipykernel_21904/2111376494.py:1: FutureWarning: In a future version of pandas all arguments of read_csv except for the argument 'filepath_or_buffer' will be keyword-only
  df, num_users, num_items = read_data_ml100k()


943 1682
training on  cuda


100%|██████████| 943/943 [22:22<00:00,  1.42s/it]


epoch 1,train_loss 69.4006,hit_rate 0.0976,auc 0.7205, time 1351.8 sec


100%|██████████| 943/943 [22:02<00:00,  1.40s/it]


epoch 2,train_loss 66.5728,hit_rate 0.1273,auc 0.7186, time 1330.5 sec
